In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/train_V2', './input/test_V2']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

# How to Score (0.0255 - 0.0245)

Before I start explaining the code, I have to say that the most of this code was taken from [[This kernel](https://www.kaggle.com/anycode/simple-nn-baseline-3)] by [anycode](https://www.kaggle.com/anycode) Kaggler

**Fisrt :** Let's load the needed dependencies :

In [1]:
import numpy as np
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])

# ALEX: remove plotting, ML code
# import seaborn as sns
# import matplotlib.pyplot as plt  

# from timeit import default_timer as timer
# from sklearn import preprocessing

# !pip install ultimate
# from ultimate.mlp import MLP 

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.callbacks import ModelCheckpoint

# import gc, sys
# gc.enable()


In [2]:
# ALEX: remove extra display code
# def state(message,start = True, time = 0):
#     if(start):
#         print(f'Working on {message} ... ')
#     else :
#         print(f'Working on {message} took ({round(time , 3)}) Sec \n')

In [3]:
INPUT_DIR = "./input/"

** I explained the following code with comments, I hope you understand it well**

In [4]:
def feature_engineering(is_train=True):
    # When this function is used for the training data, load train_V2.csv :
    if is_train: 
        print("processing train_V2.csv")
        df = pd.read_csv(INPUT_DIR + 'train_V2.scaled.csv')
        
        # Only take the samples with matches that have more than 1 player 
        # there are matches with no players or just one player ( those samples could affect our model badly) 
        df = df[df['maxPlace'] > 1]
    
    # When this function is used for the test data, load test_V2.csv :
    else:
        print("processing test_V2.csv")
        df = pd.read_csv(INPUT_DIR + 'test_V2.scaled.csv')
        
    # Make a new feature indecating the total distance a player cut :
# ALEX: remove extra display code
#     state('totalDistance')
#     s = timer()
    df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]
# ALEX: remove extra display code
#     e = timer()
#     state('totalDistance', False, e - s)
          

# ALEX: remove extra display code
#     state('rankPoints')
#     s = timer()
    # Process the 'rankPoints' feature by replacing any value of (-1) to be (0) :
    df['rankPoints'] = np.where(df['rankPoints'] <= 0 ,0 , df['rankPoints'])
# ALEX: remove extra display code
#     e = timer()                                  
#     state('rankPoints', False, e-s)
    

    target = 'winPlacePerc'
    # Get a list of the features to be used
    features = list(df.columns)
    
    # Remove some features from the features list :
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchDuration")
    features.remove("matchType")
    
    y = None
    
    # If we are processing the training data, process the target
    # (group the data by the match and the group then take the mean of the target) 
    if is_train: 
        y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
        # Remove the target from the features list :
        features.remove(target)
    
    # Make new features indicating the mean of the features ( grouped by match and group ) :
    print("get group mean feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    
    # If we are processing the training data let df_out = the grouped  'matchId' and 'groupId'
    if is_train: df_out = agg.reset_index()[['matchId','groupId']]
    # If we are processing the test data let df_out = 'matchId' and 'groupId' without grouping 
    else: df_out = df[['matchId','groupId']]
    
    # Merge agg and agg_rank (that we got before) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the max value of the features for each group ( grouped by match )
    print("get group max feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('max')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the minimum value of the features for each group ( grouped by match )
    print("get group min feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('min')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the number of players in each group ( grouped by match )
    print("get group size feature")
    agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
     
    # Merge the group_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the mean value of each features for each match :
    print("get match mean feature")
    agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
    
    # Merge the new agg with df_out :
    df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
    
    # Make new features indicating the number of groups in each match :
    print("get match size feature")
    agg = df.groupby(['matchId']).size().reset_index(name='match_size')
    
    # Merge the match_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId'])
    
    # Drop matchId and groupId
    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)
    
    # X is the output dataset (without the target) and y is the target :
    X = np.array(df_out, dtype=np.float64)
    
    
# ALEX: remove GC code
#     del df, df_out, agg, agg_rank
#     gc.collect()

    return X, y


### Process and scale the training data  :

In [5]:
# ALEX: remove IPython commands
# %%time
# Process the training data :
x_train, y = feature_engineering(True)
# Scale the data to be in the range (-1 , 1)
# ALEX: remove ML code
# scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=False).fit(x_train)

processing train_V2.csv
get group mean feature
get group max feature
get group min feature
get group size feature
get match mean feature
get match size feature


In [6]:
print("x_train", x_train.shape, x_train.max(), x_train.min())
# ALEX: remove ML code
# scaler.transform(x_train)
print("x_train", x_train.shape, x_train.max(), x_train.min())

x_train (2026744, 170) 41270.1 0.0
x_train (2026744, 170) 41270.1 0.0


### Scale the target to be in the range (-1 , 1)

In [7]:
y = y * 2 - 1
print("y", y.shape, y.max(), y.min())

y (2026744,) 1.0 -1.0


## Define an MLP model and train it :

In [8]:
# ALEX: remove IPython commands, ML code
# %%time
# # create NN_model
# NN_model = Sequential()
# NN_model.add(Dense(x_train.shape[1],  input_dim = x_train.shape[1], activation='relu'))
# NN_model.add(Dense(136, activation='relu'))
# NN_model.add(Dense(136, activation='relu'))
# NN_model.add(Dense(136, activation='relu'))
# NN_model.add(Dense(136, activation='relu'))

# # output Layer
# NN_model.add(Dense(1, activation='linear'))

# # Compile the network :
# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# NN_model.summary()
_ = x_train.shape[1]
_ = x_train.shape[1]


In [9]:
# ALEX: remove ML code
# checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
# checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
# callbacks_list = [checkpoint]


In [10]:
# ALEX: remove IPython commands, ML code
# %%time
# NN_model.fit(x=x_train, y=y, batch_size=1000,
#              epochs=30, verbose=1, callbacks=callbacks_list,
#              validation_split=0.15, validation_data=None, shuffle=True,
#              class_weight=None, sample_weight=None, initial_epoch=0,
#              steps_per_epoch=None, validation_steps=None)
# del x_train, y
# gc.collect()

## Process the test data and scale it :

In [11]:
x_test, _ = feature_engineering(False)
# ALEX: remove ML code
# scaler.transform(x_test)
print("x_test", x_test.shape, x_test.max(), x_test.min())
np.clip(x_test, out=x_test, a_min=-1, a_max=1)
print("x_test", x_test.shape, x_test.max(), x_test.min())

processing test_V2.csv
get group mean feature
get group max feature
get group min feature
get group size feature
get match mean feature
get match size feature
x_test (1934174, 170) 40799.92 0.0
x_test (1934174, 170) 1.0 0.0


### Predict the target using the test data : 

In [12]:
# ALEX: remove IPython commands, ML code
# %%time
# pred = NN_model.predict(x_test)
# del x_test
# gc.collect()

### Reshape the predictions and put them in the right range(0 , 1 )

In [13]:
# ALEX: remove ML code
# pred = pred.reshape(-1)
# pred = (pred + 1) / 2

In [14]:
df_test = pd.read_csv(INPUT_DIR + 'test_V2.scaled.csv')

In [15]:
# ALEX: remove IPython commands, ML code
# %%time
pred = [0 for _ in range(len(df_test))]
print("fix winPlacePerc")
for i in range(len(df_test)):
# ALEX: remove ML code
#     winPlacePerc = pred[i]
    winPlacePerc = 0.5
    maxPlace = int(df_test.iloc[i]['maxPlace'])
    if maxPlace == 0:
        winPlacePerc = 0.0
    elif maxPlace == 1:
        winPlacePerc = 1.0
    else:
        gap = 1.0 / (maxPlace - 1)
        winPlacePerc = round(winPlacePerc / gap) * gap
    
    if winPlacePerc < 0: winPlacePerc = 0.0
    if winPlacePerc > 1: winPlacePerc = 1.0    
    pred[i] = winPlacePerc

fix winPlacePerc


In [16]:
df_test['winPlacePerc'] = pred

## Create the submission file : 

In [17]:
submission = df_test[['Id', 'winPlacePerc']]
submission.to_csv('submission.csv', index=False)

 Finally, I hope you understand every line of this kernel, also if you have any note don't hesitate to put it in a comment .